<a href="https://colab.research.google.com/github/wawan-ikhwan/run-once-colab/blob/main/run_once_cell.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Define `run_once` cell magic
from IPython.core.magic import register_cell_magic
from IPython.display import display, HTML
from google.colab import output
from hashlib import sha1
import time
import os

@register_cell_magic
def run_once(line: str, cell: str):
  """
  This is a custom cell magic command in order to prevent multiple runs, so the cell must be run once for every runtime.
  If the cell has already run before, then it will display the history output.
  Usage: %%run_once
  """
  try:

    # Create input-output tracker directory
    if not os.path.exists('./.run_once'):
      os.makedirs('./.run_once')

    # Get cell id by cell changes with SHA-1 as identifier
    cell_id = sha1(cell.encode('utf-8')).hexdigest()

    # Check wether cell_id.html already exists. (Imply the cell already run)
    if os.path.exists(f'./.run_once/{cell_id}.html'):
      with open(f'./.run_once/{cell_id}.html', 'r') as file_cell_output:
        html_output = file_cell_output.read()
      last_created = time.strftime('%Y-%m-%d %H:%M:%S UTC+00', time.gmtime(os.path.getctime(f'./.run_once/{cell_id}.html')))
      print('This cell already ran before, last run was:', last_created, ', displaying history output...\n')
      display(HTML(html_output))
      return

    # Execute actual code.
    exec(cell, globals(), locals())

    # After code has executed, it will contains some output. Save it!
    html_output = output.eval_js('document.getElementById("output-area").innerHTML;')
    with open(f'./.run_once/{cell_id}.html', 'w') as file_cell_output:
      file_cell_output.write(html_output)

    # Also save the cell code for archive purpose
    with open(f'./.run_once/{cell_id}.py', 'w') as file_cell_input:
      file_cell_input.write('cell')

  except Exception as e:
    print(f"run_once error: {str(e)}")

In [2]:
#@title This is the first run.
%%run_once

print("This is an example of cell that shouldn't be called multiple times.")

This is an example of cell that shouldn't be called multiple times.


In [3]:
#@title This is the second run.
%%run_once

print("This is an example of cell that shouldn't be called multiple times.")

This cell already ran before, last run was: 2024-01-14 14:54:56 UTC+00 , displaying history output...

